<a href="https://colab.research.google.com/github/weasel-codes/music-classification/blob/main/Music_Genre_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATASET CREATION



Extracts MFCCs from music dataset and saves them into a json file along with genre labels.

Different variables used here are :

    :param dataset_path (str): Path to dataset

    :param json_path (str): Path to json file used to save MFCCs

    :param num_mfcc (int): Number of coefficients to extract
    
    :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples

    :param hop_length (int): Sliding window for FFT. Measured in # of samples

    :param: num_segments (int): Number of segments we want to divide sample tracks into


In [1]:
# LIBRARIES
import json
import os
import math
import librosa

In [23]:
#CONSTANTS
DATASET_PATH = "/content/drive/MyDrive/ML/songs"
JSON_PATH = "/content/drive/MyDrive/ML/MFCC.json"

SAMPLE_RATE = 22050
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION # TOTAL samples from a track


In [24]:
# PROCESS WACH SEGMENT OF AUDIO FILE SEPERATELY
def processEachSegmentOfFile(segment_num, data, file_path, curr_signal, sample_rate, num_mfcc, n_fft, hop_length, num_mfcc_vectors_per_segment, label_index):
    mfcc = librosa.feature.mfcc(curr_signal, sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length) # EXTRACT MFCC
    mfcc = mfcc.T
    if len(mfcc) == num_mfcc_vectors_per_segment:     # store only mfcc feature with expected number of vectors
        data["mfcc"].append(mfcc.tolist())
        data["labels"].append(label_index)
        print("{}, segment:{}".format(file_path, segment_num+1))

In [25]:
# PROCESS EACH AUDIO FILES SEPARATELY
def processEachAudioFile(data, dirpath, file_name, num_segments, samples_per_segment, num_mfcc, n_fft, hop_length, num_mfcc_vectors_per_segment, label_index):
    file_path = os.path.join(dirpath, file_name)  # load audio file from FULL PATH + FILE NAME
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

    for d in range(num_segments):             #process all segments of audio file : DIVIDED one file into MULTIPLE SEGMENTS
        start = samples_per_segment * d       #Start time of current segment
        finish = start + samples_per_segment  #Finish Time of segment

        curr_signal = signal[start:finish]    #Extract required samples out of signal
        processEachSegmentOfFile(d, data, file_path, curr_signal, sample_rate, num_mfcc, n_fft, hop_length, num_mfcc_vectors_per_segment, label_index)

In [29]:
# SAVE MFCC-INFO CARRYING DATA-JSON TO A FILE ON DRIVE
def saveDataToJson(data):
    with open(JSON_PATH, "w") as fp:
        json.dump(data, fp, indent=4)

In [27]:
# Dictionary to store mapping, labels, and MFCCs
data = {
    "mapping": [],  #Relate to labels
    "labels": [],   #Output
    "mfcc": []      #Training Input
}

# PROCESS EACH GENRE
def save_mfcc(num_mfcc, n_fft, hop_length, num_segments):
    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # dirpath : folder we are currently in, dirnames : sub-folders or genre, filenames : each file of a genre
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(DATASET_PATH)): # loop through all genre sub-folder

        label_index = i-1
        # ensure we're processing a genre sub-folder level : to make sure we are inside some genre : genre level and not dataset level
        if dirpath is not DATASET_PATH:

            # save genre label (i.e., sub-folder name) in the mapping
            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label) # No need if we already know what all labels we have
            print("\nProcessing: {}".format(semantic_label))

            for file_name in filenames: # Procesing each file
                processEachAudioFile(data, dirpath, file_name, num_segments, samples_per_segment, num_mfcc, n_fft, hop_length, num_mfcc_vectors_per_segment, label_index)

    saveDataToJson(data)

In [ ]:
if __name__ == "__main__":
    num_mfcc=13
    n_fft=2048
    hop_length=512
    num_segments=5
    save_mfcc(num_mfcc, n_fft, hop_length, num_segments)